In [7]:
from google.colab import drive
import sqlite3
import pandas as pd

# ربط Google Drive
drive.mount('/content/drive')

# تحديد مسار قاعدة البيانات داخل Google Drive
db_path = "/content/drive/MyDrive/publications.sqlite"

# إنشاء الاتصال بقاعدة البيانات
conn = sqlite3.connect(db_path)
cursor = conn.cursor()
print("✅ تم الاتصال بـ Google Drive وإنشاء قاعدة البيانات!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ تم الاتصال بـ Google Drive وإنشاء قاعدة البيانات!


In [8]:
# إنشاء جدول authors
cursor.execute("""
CREATE TABLE IF NOT EXISTS authors (
    au_id TEXT PRIMARY KEY,
    au_lname TEXT,
    au_fname TEXT
);
""")

# إنشاء جدول publishers
cursor.execute("""
CREATE TABLE IF NOT EXISTS publishers (
    pub_id TEXT PRIMARY KEY,
    pub_name TEXT
);
""")

# إنشاء جدول titles
cursor.execute("""
CREATE TABLE IF NOT EXISTS titles (
    title_id TEXT PRIMARY KEY,
    title TEXT,
    pub_id TEXT,
    ytd_sales INTEGER,
    FOREIGN KEY (pub_id) REFERENCES publishers(pub_id)
);
""")

# إنشاء جدول titleauthor
cursor.execute("""
CREATE TABLE IF NOT EXISTS titleauthor (
    au_id TEXT,
    title_id TEXT,
    FOREIGN KEY (au_id) REFERENCES authors(au_id),
    FOREIGN KEY (title_id) REFERENCES titles(title_id)
);
""")

conn.commit()
print("✅ تم إنشاء جميع الجداول بنجاح!")


✅ تم إنشاء جميع الجداول بنجاح!


In [9]:
# إدخال بيانات المؤلفين (authors)
authors_data = [
    ("A1", "Smith", "John"),
    ("A2", "Johnson", "Alice"),
    ("A3", "Williams", "Robert")
]
cursor.executemany("INSERT OR IGNORE INTO authors (au_id, au_lname, au_fname) VALUES (?, ?, ?);", authors_data)

# إدخال بيانات الناشرين (publishers)
publishers_data = [
    ("P1", "Penguin Books"),
    ("P2", "HarperCollins")
]
cursor.executemany("INSERT OR IGNORE INTO publishers (pub_id, pub_name) VALUES (?, ?);", publishers_data)

# إدخال بيانات العناوين (titles)
titles_data = [
    ("T1", "SQL Basics", "P1", 500),
    ("T2", "Advanced SQL", "P1", 300),
    ("T3", "Python for Data", "P2", 800)
]
cursor.executemany("INSERT OR IGNORE INTO titles (title_id, title, pub_id, ytd_sales) VALUES (?, ?, ?, ?);", titles_data)

# إدخال بيانات العناوين-المؤلفين (titleauthor)
titleauthor_data = [
    ("A1", "T1"),
    ("A1", "T2"),
    ("A2", "T3"),
    ("A3", "T1")
]
cursor.executemany("INSERT OR IGNORE INTO titleauthor (au_id, title_id) VALUES (?, ?);", titleauthor_data)

conn.commit()
print("✅ تم إدخال البيانات بنجاح!")


✅ تم إدخال البيانات بنجاح!


In [10]:
# استعلام Challenge 1 - العناوين والناشرين لكل مؤلف
query1 = """
SELECT
    a.au_id AS "AUTHOR ID",
    a.au_lname AS "LAST NAME",
    a.au_fname AS "FIRST NAME",
    t.title AS "TITLE",
    p.pub_name AS "PUBLISHER"
FROM
    authors a
JOIN
    titleauthor ta ON a.au_id = ta.au_id
JOIN
    titles t ON ta.title_id = t.title_id
JOIN
    publishers p ON t.pub_id = p.pub_id;
"""
df1 = pd.read_sql(query1, conn)
print("📌 Challenge 1:")
display(df1)

# استعلام Challenge 2 - عدد العناوين التي نشرها كل مؤلف
query2 = """
SELECT
    a.au_id AS "AUTHOR ID",
    a.au_lname AS "LAST NAME",
    a.au_fname AS "FIRST NAME",
    p.pub_name AS "PUBLISHER",
    COUNT(t.title) AS "TITLE COUNT"
FROM
    authors a
JOIN
    titleauthor ta ON a.au_id = ta.au_id
JOIN
    titles t ON ta.title_id = t.title_id
JOIN
    publishers p ON t.pub_id = p.pub_id
GROUP BY
    a.au_id, a.au_lname, a.au_fname, p.pub_name;
"""
df2 = pd.read_sql(query2, conn)
print("📌 Challenge 2:")
display(df2)

# استعلام Challenge 3 - المؤلفون الأكثر مبيعًا (أعلى 3)
query3 = """
SELECT
    a.au_id AS "AUTHOR ID",
    a.au_lname AS "LAST NAME",
    a.au_fname AS "FIRST NAME",
    SUM(t.ytd_sales) AS "TOTAL"
FROM
    authors a
JOIN
    titleauthor ta ON a.au_id = ta.au_id
JOIN
    titles t ON ta.title_id = t.title_id
GROUP BY
    a.au_id, a.au_lname, a.au_fname
ORDER BY
    "TOTAL" DESC
LIMIT 3;
"""
df3 = pd.read_sql(query3, conn)
print("📌 Challenge 3:")
display(df3)

# استعلام Challenge 4 - جميع المؤلفين مرتبين حسب المبيعات
query4 = """
SELECT
    a.au_id AS "AUTHOR ID",
    a.au_lname AS "LAST NAME",
    a.au_fname AS "FIRST NAME",
    COALESCE(SUM(t.ytd_sales), 0) AS "TOTAL"
FROM
    authors a
LEFT JOIN
    titleauthor ta ON a.au_id = ta.au_id
LEFT JOIN
    titles t ON ta.title_id = t.title_id
GROUP BY
    a.au_id, a.au_lname, a.au_fname
ORDER BY
    "TOTAL" DESC;
"""
df4 = pd.read_sql(query4, conn)
print("📌 Challenge 4:")
display(df4)


📌 Challenge 1:


,AUTHOR ID,LAST NAME,FIRST NAME,TITLE,PUBLISHER
0,A01,Smith,John,Book A,Penguin Books
1,A02,Johnson,Emily,Book B,HarperCollins
2,A03,Williams,Michael,Book C,Random House
3,A01,Smith,John,Book C,Random House
4,A01,Smith,John,Book A,Penguin Books
5,A02,Johnson,Emily,Book B,HarperCollins
6,A03,Williams,Michael,Book C,Random House
7,A01,Smith,John,Book C,Random House
8,A1,Smith,John,SQL Basics,Penguin Books
9,A1,Smith,John,Advanced SQL,Penguin Books


📌 Challenge 2:


,AUTHOR ID,LAST NAME,FIRST NAME,PUBLISHER,TITLE COUNT
0,A01,Smith,John,Penguin Books,2
1,A01,Smith,John,Random House,2
2,A02,Johnson,Emily,HarperCollins,2
3,A03,Williams,Michael,Random House,2
4,A1,Smith,John,Penguin Books,2
5,A2,Johnson,Alice,HarperCollins,1
6,A3,Williams,Robert,Penguin Books,1


📌 Challenge 3:


,AUTHOR ID,LAST NAME,FIRST NAME,TOTAL
0,A01,Smith,John,2400
1,A03,Williams,Michael,1400
2,A1,Smith,John,800


📌 Challenge 4:


,AUTHOR ID,LAST NAME,FIRST NAME,TOTAL
0,A01,Smith,John,2400
1,A03,Williams,Michael,1400
2,A1,Smith,John,800
3,A2,Johnson,Alice,800
4,A02,Johnson,Emily,600
5,A3,Williams,Robert,500


In [11]:
sql_file_path = "/content/drive/MyDrive/solutions.sql"

solutions_sql = f"""
{query1}

{query2}

{query3}

{query4}
"""

with open(sql_file_path, "w") as file:
    file.write(solutions_sql)

print(f"✅ تم حفظ الملف في Google Drive: {sql_file_path}")


✅ تم حفظ الملف في Google Drive: /content/drive/MyDrive/solutions.sql
